# G5HT-PIPELINE

first transfers data from flvc to local directory, then moves results back to flvc

The transferring code assumes you have set up an ssh key

__set up ssh key__

https://github.com/flavell-lab/FlavellLabWiki/wiki/Setting-up-SSH-key-on-Windows
- generate ssh key
- add to github account
- transfer authorized keys to flvc
  - type $env:USERPROFILE\.ssh\id_rsa.pub | ssh munib@flv-c3 "mkdir -p ~/.ssh && cat >> ~/.ssh/authorized_keys"
- done, should be able to ssh into flvc (ssh flvc or ssh munib@flv-c3) without entering password

for transferring back, probably better to use rsync in msys2

## CONDA ENVIRONMENTS

For steps __transfer files__, __1. preprocess__ and __2. mip__, `conda activate g5ht-pipeline`

For step __3. segment__, `conda activate segment-torch` or `conda activate torchcu129`

For step __4. spline, 5. orient, 6. warp, 7. reg__, `conda activate g5ht-pipeline`


## IMPORTS

In [ ]:
import sys
import os
import importlib
from tqdm import tqdm
from pathlib import Path
from datetime import datetime

try:
    import utils
    is_torch_env = False
except ImportError:
    is_torch_env = True
    print("utils not loaded because conda environment doesn't have nd2reader installed. probably using torchcu129 env, which is totally fine for just doing the segmentation step")

## TRANSFER FILES

Takes about 20 mins

In [3]:
flvc = 'munib@flv-c3' # this is the username and hostname of the linux machine
data_dir_flvc = r'/home/munib/store1/shared/g5ht/data' # this is a linux machine
data_dir_local = Path(r'C:\Users\munib\POSTDOC\DATA\g5ht-free') # this is a windows machine

# dataset (see datasets.txt)
dataset = 'date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.nd2'

In [ ]:

# use subprocess and scp to copy the file from the linux machine to the windows machine
# additionally will transfer a file with same name but with .h5 extension and a file with same name, but with ...wormXXX _chan_alignment.nd2 as filename

# def scp_from_flvc(filename, data_dir_flvc, data_dir_local, flvc):
    
#     # 1. Handle Local Path (Standard Windows style)
#     date_str = filename.split('_')[0].split('-')[1]
#     local_dir = data_dir_local / date_str
#     local_dir.mkdir(parents=True, exist_ok=True)

#     # 2. Handle Remote Path (Force Linux/Posix style)
#     remote_path = PurePosixPath(data_dir_flvc) / date_str / filename

#     # 3. Check if remote file exists
#     ssh_command = f'ssh {flvc} "test -e {remote_path}"'
#     print(ssh_command)
    
#     result = subprocess.run(ssh_command, shell=True, capture_output=True, text=True)
#     if result.returncode != 0:
#         print(f"Remote file not found or error: {result.stderr}")
#         return

#     # 4. Transfer using scp (available on Windows via OpenSSH)
#     print(f"Transferring {filename} to {local_dir}...")
#     scp_command = f'scp "{flvc}:{remote_path}" "{local_dir}"'
#     print(scp_command)
#     subprocess.run(scp_command, shell=True, check=True)

# transfer the files
utils.scp_from_flvc(dataset, data_dir_flvc, data_dir_local, flvc) # ~10 mins from flvc to local windows
utils.scp_from_flvc(os.path.splitext(dataset)[0] + '.h5', data_dir_flvc, data_dir_local, flvc) # ~ 3 mins
utils.scp_from_flvc(os.path.splitext(dataset)[0] + '_chan_alignment.nd2', data_dir_flvc, data_dir_local, flvc) # ~ 3 mins

ssh munib@flv-c3 "test -e /home/munib/store1/shared/g5ht/data/20251028/date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.nd2"
Transferring date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.nd2 to C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028...
scp "munib@flv-c3:/home/munib/store1/shared/g5ht/data/20251028/date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.nd2" "C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028"
ssh munib@flv-c3 "test -e /home/munib/store1/shared/g5ht/data/20251028/date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.h5"
Transferring date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.h5 to C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028...
scp "munib@flv-c3:/home/munib/store1/shared/g5ht/data/20251028/date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.h5" "C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028"
ssh munib@flv-c3 "test -e /home/munib/store1/shared/g5ht/data/2

## SPECIFY DATA TO PROCESS

In [ ]:
INPUT_ND2 = dataset
date_str = INPUT_ND2.split('_')[0].split('-')[1]
local_dir = data_dir_local / date_str
local_dir.mkdir(parents=True, exist_ok=True)
INPUT_ND2_PTH = os.path.join(local_dir, INPUT_ND2)

NOISE_PTH = r'C:\Users\munib\POSTDOC\CODE\g5ht-pipeline\noise\noise_111125.tif'

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 41 if 'immo' not in INPUT_ND2 else 122

# for recordings prior to roughly December 2025, we want to keep all but the last two z-slices, during which the piezo position is unstable
# after December 2025, we want to keep all but the first two z-slices, during which the piezo position is unstable at the beginning of the recording
date_obj = datetime.strptime(date_str, '%Y%m%d')
if date_obj < datetime(2025, 12, 1):
    z2keep =  (0,STACK_LENGTH-2) # tuple representing range of z-slices to keep, should keep all but the last two slices
else:
    z2keep =  (2,STACK_LENGTH) # tuple representing range of z-slices to keep, should keep all but the first two slices

if not is_torch_env:
    noise_stack = utils.get_noise_stack(NOISE_PTH, STACK_LENGTH)
    num_frames, height, width, num_channels = utils.get_range_from_nd2(INPUT_ND2_PTH, stack_length=STACK_LENGTH) 
    beads_alignment_file = utils.get_beads_alignment_file(INPUT_ND2_PTH)
else:
    print("utils not loaded because conda environment doesn't have nd2reader installed. probably using torchcu129 env, which is totally fine for just doing the segmentation step")

print(INPUT_ND2)
print('Num z-slices: ', STACK_LENGTH)
if not is_torch_env:
    print('Number of frames: ', num_frames)
    print('Height: ', height)
    print('width: ', width)
    print('Number of channels: ', num_channels)
    print('Beads alignment file: ', beads_alignment_file)

date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.nd2
Num z-slices:  41
Number of frames:  1200
Height:  512
width:  512
Number of channels:  2
Beads alignment file:  None


## 1. SHEAR CORRECTION 

` conda activate g5ht-pipeline`

~ 1 hour for 1200 image stacks with 2 color channels, 41 z, 512 h, 512 w

- shear corrects each volume
  - depending on each exposure time, it can take roughly half a second between the first and last frames of a volume, so any movements need to be corrected for
- creates one `.tif` for each volume and stores it in the `shear_corrected` directory

In [ ]:
import shear_correct
_ = importlib.reload(sys.modules['shear_correct'])

start_index = "0"
end_index = str(num_frames-1)
# start_index = "59"
# end_index = "803"
cpu_count = str(int(os.cpu_count() / 2))
# cpu_count = str(int(os.cpu_count()))
skip_shear_correction = False # if True, will just denoise and save as tif

# sys.argv = ["", nd2 file, start_frame, end_frame, noise_pth, stack_length, n_workers, num_frames, height, width, num_channels]
sys.argv = ["", INPUT_ND2_PTH, start_index, end_index, NOISE_PTH, STACK_LENGTH, cpu_count, num_frames, height, width, num_channels, z2keep, skip_shear_correction]

# Call the main function
shear_correct.main()

Processing 1141 stacks (59-1199) using 10 workers...


 43%|████▎     | 490/1141 [26:19<34:33,  3.19s/it]  

## 2. CHANNEL ALIGNMENT

` conda activate g5ht-pipeline`

### 2a. GET MEDIAN CHANNEL ALIGNMENT PARAMETERS FROM ALL FRAMES

- If channel alignment file found, uses that, if not uses worm recording
- creates a `.txt` file for each volume that contains elastix channel registration parameters
- creates `chan_align_params.csv` and  `chan_align.txt`

In [ ]:
beads_alignment_file

In [ ]:
import get_channel_alignment
import median_channel_alignment
_ = importlib.reload(sys.modules['get_channel_alignment'])
_ = importlib.reload(sys.modules['median_channel_alignment'])

## set beads_alignment_file to None to use worm recording for channel alignment, even if beads file exists
# beads_alignment_file = None

start_index = "0"
cpu_count = str(int(os.cpu_count() / 2))
# cpu_count = str(int(os.cpu_count()))

if beads_alignment_file is not None:
    align_with_beads = True
    num_frames_beads, _, _, _ = utils.get_range_from_nd2(beads_alignment_file, stack_length=STACK_LENGTH) 
    sys.argv = ["", beads_alignment_file, start_index, str(num_frames_beads-1), NOISE_PTH, STACK_LENGTH, cpu_count, num_frames_beads, height, width, num_channels, align_with_beads]
else:
    align_with_beads = False
    sys.argv = ["", INPUT_ND2_PTH, start_index, str(num_frames-1), NOISE_PTH, STACK_LENGTH, cpu_count, num_frames, height, width, num_channels, align_with_beads]

# # Call the main function
get_channel_alignment.main()
median_channel_alignment.main()


### 2b. APPLY MEDIAN CHANNEL ALIGNMENT PARAMETERS

- ouputs aligned volumes in `channel_aligned` directory

In [ ]:
import apply_channel_alignment
_ = importlib.reload(sys.modules['apply_channel_alignment'])

start_index = "0"
cpu_count = str(int(os.cpu_count() / 2))
# cpu_count = str(int(os.cpu_count()))

# 0786 to 0799 are bad frames in worm005.nd2, copied 0785 for each of those frames

if beads_alignment_file is not None:
    align_with_beads = True
    num_frames_beads, _, _, _ = utils.get_range_from_nd2(beads_alignment_file, stack_length=STACK_LENGTH) 
    sys.argv = ["", INPUT_ND2_PTH, start_index, str(num_frames-1), NOISE_PTH, STACK_LENGTH, cpu_count, num_frames, height, width, num_channels, align_with_beads, beads_alignment_file]
else:
    align_with_beads = False
    sys.argv = ["", INPUT_ND2_PTH, start_index, str(num_frames-1), NOISE_PTH, STACK_LENGTH, cpu_count, num_frames, height, width, num_channels, align_with_beads]


# Call the main function
apply_channel_alignment.main()

### 2c. PLOT CHANNEL ALIGNMENT PARAMETER DISTRIBUTIONS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# make font sizes larger for visibility
plt.rcParams.update({'font.size': 18})

try:
    out_dir = os.path.splitext(INPUT_ND2_PTH)[0]

    df = pd.read_csv(os.path.join(out_dir, 'chan_align_params.csv'))
    params = ['TransformParameter_0', 'TransformParameter_1', 'TransformParameter_2', 'TransformParameter_3', 'TransformParameter_4', 'TransformParameter_5']
    labels = ['Rx', 'Ry', 'Rz', 'Tx', 'Ty', 'Tz']

    # the xaxis limits for each subplot should be the same across figures

    xlims = np.zeros((6,2))

    plt.figure(figsize=(12,8), tight_layout=True)
    for i,param in enumerate(params):
        plt.subplot(2, 3, i+1)
        plt.hist(df[param], bins=30, color='red', alpha=0.6)
        # plot the median value as a vertical line
        median_value = df[param].median()
        plt.axvline(median_value, color='black', linestyle='dashed', linewidth=2)
        plt.xlabel(labels[i])
        plt.ylabel('Frequency')
        # get xaxis limits
        xlims[i,:] = plt.xlim()
        # title is median value
        plt.title(f'Median: {np.round(median_value,3)}', fontsize=14)
    plt.show()
except FileNotFoundError:
    print("No chan_align_params.csv found for worm recording")

out_dir = os.path.splitext(INPUT_ND2_PTH)[0] + '_chan_alignment'
df = pd.read_csv(os.path.join(out_dir, 'chan_align_params.csv'))
params = ['TransformParameter_0', 'TransformParameter_1', 'TransformParameter_2', 'TransformParameter_3', 'TransformParameter_4', 'TransformParameter_5']
labels = ['Rx', 'Ry', 'Rz', 'Tx', 'Ty', 'Tz']

plt.figure(figsize=(12,8), tight_layout=True)
for i,param in enumerate(params):
    plt.subplot(2, 3, i+1)
    plt.hist(df[param], bins=30, color='blue', alpha=0.6)
    # plot the median value as a vertical line
    median_value = df[param].median()
    plt.axvline(median_value, color='black', linestyle='dashed', linewidth=2)
    plt.xlabel(labels[i])
    plt.ylabel('Frequency')
    # apply xlims
    # plt.xlim(xlims[i,0], xlims[i,1])
    # title is median value, font size 14
    plt.title(f'Median: {np.round(median_value,3)}', fontsize=14)
plt.show()

## 3. BLEACH CORRECTION

In [ ]:
import importlib
import os
import sys

import bleach_correct
_ = importlib.reload(sys.modules['bleach_correct'])


PTH = os.path.splitext(INPUT_ND2_PTH)[0]
REG_DIR = 'channel_aligned' # 'channel_aligned' or 'tif' 
channels = 1
method = 'block' # 'block' or 'exponential'
mode = 'total' # 'total' or 'median'
output_dir = os.path.join(PTH, 'bleach_corrected')

bleach_correct.correct_bleaching(os.path.join(PTH,REG_DIR), output_dir=output_dir, channels=channels, method=method, fbc=0.04, intensity_mode=mode)


# # Correct RFP only with block method (default)
# correct_bleaching("path/to/data")

# # Correct both channels with exponential fit
# correct_bleaching("path/to/data", channels=[0, 1], method='exponential')

# # Command line
# python bleach_correct.py path/to/data --channels 0 1 --method exponential

## 4. MIP

` conda activate g5ht-pipeline`

- outputs `means.png`, `focus.png`, `mip.tif`, and `mip.mp4`, `focus_check.csv`

##### TODO: 
- legend for focus.png, should be frame#
- mip for xy, xz, zy
- mip for several slices

In [ ]:
import mip

_ = importlib.reload(sys.modules['mip'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
framerate = 8
# tif_dir = 'bleach_corrected_RFP_block' # one of 'shear_corrected' 'channel_aligned' 'bleach_corrected_RFP_block'
tif_dir = 'shear_corrected'
rmax = 850
gmax = 150
mp4_quality = 10
do_focus = True
sys.argv = ["", INPUT_ND2_PTH, tif_dir, STACK_LENGTH, num_frames, framerate, rmax, gmax, mp4_quality, do_focus]

# Call the main function
mip.main()

## 5 DRIFT ESTIMATION

` conda activate g5ht-pipeline`

- outputs  `z_selection.csv`, `z_selection_diagnostics.png`, `sharpness.csv`

TODO:
- use z selection going forward
- also use sharpness/focus (and other things) to determine good/bad frames

In [ ]:
import drift_estimation

_ = importlib.reload(sys.modules['drift_estimation'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
tif_dir = 'bleach_corrected_RFP_block' # one of 'shear_corrected' 'channel_aligned' 'bleach_corrected_RFP_block'

sys.argv = ["", INPUT_ND2_PTH, tif_dir, STACK_LENGTH, num_frames]

# Call the main function
drift_estimation.main()

## 5. SEGMENT

- outputs `label.tif`, contains segmented MIP for each volume

__on home pc__: 
`conda activate segment-torch`

Uses a separate conda environment from the rest of the pipeline. create it using:
`conda env create -f segment_torch.yml`

__on lab pc__: 
`conda activate torchcu129`

Uses a separate conda environment from the rest of the pipeline. create it following steps in:
`segment_torch_cu129_environment.yml`

### setup each time model weights change
Need to set path to model weights as `CHECKPOINT` in `eval_torch.py`

In [ ]:
import segment.segment_torch
_ = importlib.reload(sys.modules['segment.segment_torch'])

# mip_tif = 'mip_bleach_corrected_RFP_block'
mip_tif = 'mip_channel_aligned' 

MIP_PTH = os.path.join(os.path.splitext(INPUT_ND2_PTH)[0], f'{mip_tif}.tif')

# command-line arguments
sys.argv = ["", MIP_PTH]

segment.segment_torch.main()

## 6. SPLINE

`conda activate g5ht-pipeline`

- outputs `spline.json`, `spline.tif`, and `dilated.tif`

In [ ]:
import spline
_ = importlib.reload(sys.modules['spline'])

LABEL_PTH = MIP_PTH = os.path.join(os.path.splitext(INPUT_ND2_PTH)[0], 'label.tif')

# command-line arguments
sys.argv = ["", LABEL_PTH]

spline.main()

## 7. ORIENT

`conda activate g5ht-pipeline`

- outputs `oriented.json`, `oriented.png`, `oriented_stack.tif`

NOTE: `orient_v2.py` automated the process of finding orientation completely, whereas `orient.py` requires you to input the (x,y) nose location on the first frame

In [ ]:
import orient
_ = importlib.reload(sys.modules['orient'])

SPLINE_PTH = MIP_PTH = os.path.join(os.path.splitext(INPUT_ND2_PTH)[0], 'spline.json')
nose_y = 250
nose_x = 45

# apply constraints
# might need this when there are frames where the spline fitting fails and orientation is lost intermittently
constrain_frame = 515
constrain_frame_nose_y = 288
constrain_frame_nose_x = 180

# command-line arguments
# sys.argv = ["", SPLINE_PTH, str(nose_y), str(nose_x)]
sys.argv = ["", SPLINE_PTH, str(nose_y), str(nose_x), str(constrain_frame), str(constrain_frame_nose_y), str(constrain_frame_nose_x)]

orient.main()

In [ ]:
import orient_v2 # tried to automate finding nose point, not working well at the moment
_ = importlib.reload(sys.modules['orient_v2'])

SPLINE_PTH = MIP_PTH = os.path.join(os.path.splitext(INPUT_ND2_PTH)[0], 'spline.json')

# command-line arguments
sys.argv = ["", SPLINE_PTH]

orient_v2.main()

## 8. WARP

`conda activate g5ht-pipeline`

- ouputs: `warped/*.tif` and `masks/*.tif`

TODO: parallelize

In [ ]:
import warp
_ = importlib.reload(sys.modules['warp'])

PTH = os.path.splitext(INPUT_ND2_PTH)[0]

start_index = 516
end_index = num_frames

for i in tqdm(range(start_index, end_index)):
    # command-line arguments
    sys.argv = ["", PTH, i]

    warp.main()

## 9. REGISTER

`conda activate g5ht-pipeline`

__ALTERNATIVELY__: register using the wholistic registration algorithm, currently in MATLAB

TODO: parallelize / make faster

- pick a good representative fixed frame that you want to register everything to
  - copy it to the main output folder and name it `fixed_xxxx.tif`
  - copy the corresponding mask and name it `fixed_mask_xxxx.tif`

In [ ]:
import reg
_ = importlib.reload(sys.modules['reg'])

PTH = os.path.splitext(INPUT_ND2_PTH)[0]

start_index = 222
end_index = num_frames
zoom = 1 # albert was using 3
# zoom = 3

for i in tqdm(range(start_index, end_index)):
    # command-line arguments
    try:
        sys.argv = ["", PTH, i, str(zoom)]
        reg.main()
    except Exception as e:
        print(f"Error processing index {i}: {e}")   

### REGISTER WITH GFP+1 TO RFP

TRIM LAST RFP ZSLICE, TRIM FIRST GFP ZSLICE

seems to be that as of 20251204, all recordings were taken such that the i zslice in red channel corresponds to i+1 zslice in green channel

In [ ]:
import sys
import os
from tqdm import tqdm
import importlib

from reg_gfp_indexing import main as reg_worm

PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028\date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001_aligned'

for i in tqdm(range(1200)):
    # command-line arguments
    sys.argv = ["", PTH, i, "1"]
    reg_worm()

### MAKE MOVIES OF REGISTERED DATA (see `reg_microfilm.ipynb`)

### REGISTER SINGLE FRAMES WITH ERROR LOGGING

In [ ]:
import tifffile
import numpy as np
import scipy.ndimage as ndi
import itk
import sys
import os
import glob

from itk import image_view_from_array

#get channels out of stacks
def register_one(fixed_stack, fixed_mask_stack, moving_stack, moving_mask_stack):
	fixed_rfp = fixed_stack[:, 1].copy()
	moving_gfp, moving_rfp = moving_stack[:, 0].copy(), moving_stack[:, 1].copy()

	#initialize registration parameters
	parameter_object = itk.ParameterObject.New()
	default_rigid_parameter_map = parameter_object.GetDefaultParameterMap('rigid', 4)
	parameter_object.AddParameterMap(default_rigid_parameter_map)
	default_affine_parameter_map = parameter_object.GetDefaultParameterMap('affine', 4)
	parameter_object.AddParameterMap(default_affine_parameter_map)
	default_bspline_128_parameter_map = parameter_object.GetDefaultParameterMap('bspline', 4, 128)
	parameter_object.AddParameterMap(default_bspline_128_parameter_map)
	default_bspline_64_parameter_map = parameter_object.GetDefaultParameterMap('bspline', 4, 64)
	parameter_object.AddParameterMap(default_bspline_64_parameter_map)
	default_bspline_32_parameter_map = parameter_object.GetDefaultParameterMap('bspline', 4, 32)
	parameter_object.AddParameterMap(default_bspline_32_parameter_map)

	#convert to itk images
	fixed_rfp = itk.image_view_from_array(fixed_rfp.astype(np.float32))
	moving_rfp = itk.image_view_from_array(moving_rfp.astype(np.float32))

	fixed_mask_stack = itk.image_view_from_array(fixed_mask_stack.astype(np.ubyte))
	moving_mask_stack = itk.image_view_from_array(moving_mask_stack.astype(np.ubyte))

	#register rfp first and then apply transform to gfp

	registered_rfp, transform_parameters = itk.elastix_registration_method(fixed_rfp, moving_rfp, parameter_object,
																		   fixed_mask=fixed_mask_stack, moving_mask=moving_mask_stack,
																		   log_to_console=True)
	registered_gfp = itk.transformix_filter(moving_gfp, transform_parameters)

	#initialize and fill output
	output_stack = np.zeros((fixed_stack.shape[0], 2, 200, 500), np.uint16)
	output_stack[:, 0] = np.clip(registered_gfp, 0, 4095)
	output_stack[:, 1] = np.clip(registered_rfp, 0, 4095)

	return output_stack
    
	# # enablle elastic error logging
	# elastix_filter = itk.ElastixRegistrationMethod.New(fixed_rfp, moving_rfp, parameter_object,
	# 																	   fixed_mask=fixed_mask_stack, moving_mask=moving_mask_stack,
	# 																	   log_to_console=True) 
    
	# elastix_filter.SetParameterObject(parameter_object)
	# elastix_filter.SetNumberOfThreads(8)
	# elastix_filter.LogToConsoleOn()  # Enable console logging
	# elastix_filter.LogToFileOn()
	# elastix_filter.SetOutputDirectory(r"C:\Users\munib\POSTDOC\CODE\g5ht-pipeline\logs")
	# elastix_filter.Update()

	# return elastix_filter.GetOutput(), elastix_filter.GetTransformParameterObject()

input_dir = os.path.splitext(INPUT_ND2_PTH)[0]
warped_path = os.path.join(input_dir, 'warped')
output_path = os.path.join(input_dir, 'registered_fixed_sweep')
os.makedirs(output_path, exist_ok=True)

fixed_list = np.arange(0, 400, 50)
mov_list = np.arange(0, 400, 30)

for fixed in fixed_list:
    for mov in mov_list:
        print(f'Processing fixed: {fixed}, moving: {mov}')
        
		# load stacks
	
        moving_path = os.path.join(warped_path,f'{mov:04d}.tif')
        moving_stack = tifffile.imread(moving_path).astype(np.float32)
        fixed_path = os.path.join(warped_path,f'{fixed:04d}.tif')
        fixed_stack = tifffile.imread(fixed_path).astype(np.float32)
        fixed_mask_path = os.path.join(input_dir, 'masks', f'{fixed:04d}.tif')
        fixed_mask = tifffile.imread(fixed_mask_path)
        fixed_mask_stack = np.stack([fixed_mask] * fixed_stack.shape[0])

        moving_mask_path = os.path.join(input_dir, 'masks', f'{mov:04d}.tif')
        moving_mask = tifffile.imread(moving_mask_path)
        moving_mask_stack = np.stack([moving_mask] * fixed_stack.shape[0])


        output_stack = register_one(fixed_stack, fixed_mask_stack, moving_stack, moving_mask_stack)
        # save output stack with fixed and moving indices in filename
        output_file_path = os.path.join(output_path, f'fixed_{fixed:04d}_mov_{mov:04d}.tif')
        tifffile.imwrite(output_file_path, output_stack, imagej=True)

# fixed = 100
# mov = 200

# # load stacks
# moving_path = os.path.join(warped_path,f'{mov:04d}.tif')
# moving_stack = tifffile.imread(moving_path).astype(np.float32)
# fixed_path = os.path.join(warped_path,f'{fixed:04d}.tif')
# fixed_stack = tifffile.imread(fixed_path).astype(np.float32)
# fixed_mask_path = os.path.join(input_dir, 'masks', f'{fixed:04d}.tif')
# fixed_mask = tifffile.imread(fixed_mask_path)
# fixed_mask_stack = np.stack([fixed_mask] * fixed_stack.shape[0])

# moving_mask_path = os.path.join(input_dir, 'masks', f'{mov:04d}.tif')
# moving_mask = tifffile.imread(moving_mask_path)
# moving_mask_stack = np.stack([moving_mask] * fixed_stack.shape[0])


# output_stack = register_one(fixed_stack, fixed_mask_stack, moving_stack, moving_mask_stack)

In [ ]:
output_stack = register_one(fixed_stack, fixed_mask_stack, moving_stack, moving_mask_stack)

In [ ]:
output_stack.shape

In [ ]:

import matplotlib.pyplot as plt

plt.figure()
plt.pcolormesh(fixed_stack[10,1,:,:])
plt.show()

plt.figure()
plt.pcolormesh(output_stack[10,1,:,:])
plt.show()

## COPY RESULTS TO FLVC

In [ ]:
import subprocess
import os
from pathlib import Path, PurePosixPath

# --- CONFIGURATION ---
# Replace with the actual path to your MSYS2 rsync executable
RSYNC_EXE = r"C:\msys64\usr\bin\rsync.exe" 

flvc = 'munib@flv-c3'
# Remote base (Linux)
data_dir_remote_base = PurePosixPath('/home/munib/store1/shared/g5ht/data')
# Local base (Windows)
data_dir_local_base = Path(r'C:\Users\munib\POSTDOC\DATA\g5ht-free')

dataset = 'date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.nd2'

def to_msys_path(win_path):
    """
    Converts a Windows Path (C:\Users\...) to MSYS2 format (/c/Users/...).
    """
    p = Path(win_path).absolute()
    drive = p.drive.replace(":", "").lower()
    # Join parts skipping the drive letter
    parts = "/".join(p.parts[1:])
    return f"/{drive}/{parts}"

def sync_dataset_group(base_filename):
    # 1. Extract date from filename (e.g., '20251028')
    try:
        date_str = base_filename.split('_')[0].split('-')[1]
    except IndexError:
        print(f"Could not parse date from {base_filename}. Check naming convention.")
        return

    # 2. Define the three files to transfer
    file_list = [
        base_filename,
        base_filename.replace('.nd2', '.h5'),
        base_filename.replace('.nd2', '_chan_alignment.nd2')
    ]

    # 3. Setup Local Directory
    local_target_dir = data_dir_local_base / date_str
    local_target_dir.mkdir(parents=True, exist_ok=True)
    
    # Convert local target to MSYS2 format for the rsync command
    local_dest_msys = to_msys_path(local_target_dir)

    print(f"--- Starting Sync for Date: {date_str} ---")

    for file in file_list:
        # Construct Remote Path (Linux uses forward slashes)
        # Assuming the files are inside a date subfolder on Linux too:
        remote_file_path = data_dir_remote_base / date_str / file
        
        # Build rsync source string
        rsync_src = f"{flvc}:{remote_file_path}"

        # 4. Execute Rsync
        # -a: archive, -v: verbose, -z: compress, -P: progress/partial
        print(f"Syncing: {file}...")
        
        cmd = [
            RSYNC_EXE, 
            "-avzP", 
            "-e", "ssh -o StrictHostKeyChecking=no", # Uses SSH for the tunnel
            rsync_src, 
            local_dest_msys
        ]

        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode == 0:
            print(f"Successfully synced {file}")
        else:
            # If rsync fails, it might be because the file doesn't exist
            if "No such file" in result.stderr:
                print(f"File not found on remote: {file}")
            else:
                print(f"Error syncing {file}: {result.stderr}")

if __name__ == "__main__":
    sync_dataset_group(dataset)